In [1]:
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

/home/ashish/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [3]:
# Loading the architecture using imagenet as weights
MobileNetV2 = MobileNetV2(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (200, 200, 3))

/home/ashish/anaconda3/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [4]:
# Making all the layer untrainable
for layer in MobileNetV2.layers:
    layer.trainable = False

In [5]:
def definingModel(bottom_model, num_classes=11):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [6]:
num_classes = 11

FC_Head = definingModel(MobileNetV2, num_classes)

model = Model(inputs = MobileNetV2.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 201, 201, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 100, 100, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 100, 100, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [7]:
train_data_dir = './ashish/train/'
validation_data_dir = './ashish/validation/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='categorical')

Found 1077 images belonging to 11 classes.
Found 223 images belonging to 11 classes.


In [8]:
checkpoint = ModelCheckpoint("face_recognition.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1077
nb_validation_samples = 223

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
67/67 [==============================] - 123s 2s/step - loss: 1.7629 - accuracy: 0.3572 - val_loss: 1.8390 - val_accuracy: 0.3783

Epoch 00001: val_loss improved from inf to 1.83898, saving model to face_recognition.h5
Epoch 2/5
67/67 [==============================] - 111s 2s/step - loss: 1.2814 - accuracy: 0.5372 - val_loss: 2.1874 - val_accuracy: 0.3043

Epoch 00002: val_loss did not improve from 1.83898
Epoch 3/5
67/67 [==============================] - 117s 2s/step - loss: 1.1842 - accuracy: 0.5669 - val_loss: 2.6546 - val_accuracy: 0.2874

Epoch 00003: val_loss did not improve from 1.83898
Epoch 4/5
67/67 [==============================] - 108s 2s/step - loss: 1.0297 - accuracy: 0.6192 - val_loss: 2.1749 - val_accuracy: 0.3816
Restoring model weights from the end of the best epoch

Epoch 00004: val_loss did not improve from 1.83898
Epoch 00004: early stopping


In [9]:
classifier = load_model('face_recognition.h5')

In [10]:
monkey_breeds_dict = {"[0]": "Ashish Singh ", 
                      "[1]": "Brad",
                      "[2]": "Daniel",
                      "[3]": "Elon",
                      "[4]": "Ema ",
                      "[5]": "Emilia",
                      "[6]": "Maisie",
                      "[7]": "Obama",
                      "[8]": "Tom",
                      "[9]": "Trump",
                      "[10]": "Zuckerberg"}

monkey_breeds_dict_n = {"n0": "Ashish Singh ", 
                      "n1": "Brad",
                      "n2": "Daniel",
                      "n3": "Elon",
                      "n4": "Ema ",
                      "n5": "Emilia",
                      "n6": "Maisie",
                      "n7": "Obama",
                      "n8": "Tom",
                      "n9": "Trump",
                       "n10": "Zuckerberg"}

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print(path_class)
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    print(file_path+"/"+image_name)
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("./ashish/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (200, 200), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,200,200,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Brad
./ashish/validation/Brad/images - 2019-08-15T182645.211.jpg
Tom
./ashish/validation/Tom/images (82).jpg
Trump
./ashish/validation/Trump/images - 2019-08-15T151818.843.jpg
Daniel
./ashish/validation/Daniel/images - 2019-08-15T085448.198.jpg
Elon
./ashish/validation/Elon/images (73).jpg
Elon
./ashish/validation/Elon/images (96).jpg
Ema
./ashish/validation/Ema/2Q__ (2).jpg
Tom
./ashish/validation/Tom/images (69).jpg
Zuckerberg
./ashish/validation/Zuckerberg/images (71).jpeg
Trump
./ashish/validation/Trump/images - 2019-08-15T151821.156.jpg
